In [7]:
import csv
import re
import PyPDF4

In [8]:
depopath = './depos/'
csvpath = './csvs/'

filename = '05-07-21 Realtime Transcript.txt'

In [9]:
filepath = depopath + filename
outpath = csvpath + filename

In [10]:
# if .txt, readlines()
# if .pdf, convert PDF to text
if (filepath.split('.')[-1] == "txt"):
    depo = open(filepath, 'r', encoding='utf-16-le')
    lines = depo.readlines()
else:
    depo = open(filepath, 'rb')
    pdfReader = PyPDF4.PdfFileReader(depo)
    
    # extracting text from page
    lines = []

    for i in range(pdfReader.numPages):
        # ignore last 3 lines, per veritext standard (page #, veritext, phone #)
        lines.extend(pdfReader.getPage(i).extractText().splitlines()[:-3])

depo.close()

In [25]:
# Regex pattern for timestamp to remove
r = '[0-9]+:[0-9][0-9]:[0-9][0-9][A-Z][A-Z] *[0-9]*'

# 'Higly Confidential Attorney's Eyes Only' statement or 'Page ##' match string
eyes =  ["\*\*", "Page[s]* [0-9]+", "DIANE E. SKILLMAN"]

# Questioner list, includes all speakers that are not answering
qs = ["Q.\t", "Q\.", "Q\.  ", "Q  ", "[A-Z]+\.* [A-Z]+: *"]

# Responder list
ans = ["A\.\t", "A\.", "A\. ", "A  ", "THE WITNESS: *", "THE DEPONENT: *"]

In [26]:
# Leading characters to ignore
leading_chars = 0

#for line in lines:
#    if (line.strip()) and re.search("1\t", line):
#        leading_chars = re.search("1\t", line).start() + 2
#        break

In [27]:
# Speaker identifier and sublist indexer
cur_speaker = -1

# Examination start marker
started = 0

# Output in the format of [[Q,A], [Q,A], ...] **A may be empty
out_list = [["","",""]]

# What part of testimony?
# 1 = direct
# 2 = cross
# 3 = redirect
# 4 = recross
part = 0
part_dict = {
    0 : 'PRE-EXAMINATION',
    1 : 'DIRECT',
    2 : 'CROSS',
    3 : 'REDIRECT',
    4 : 'RECROSS',
    5 : 'COURT'
}

# Main loop
for line in lines:
    # Clean line:
    #   remove leading_chars
    #   regex match and remove timestamp
    #   strip() leading and tailing spaces
    line = line[leading_chars:]
    line = re.sub(r,'',line).strip()

    if 'EXAMINATION' in line:
        started = 1
        print(started)
        if 'REDIRECT' in line:
            part = 3
        elif 'RECROSS' in line:
            part = 4
        elif 'DIRECT' in line:
            part = 1
        elif 'CROSS' in line:
            part = 2
        elif 'the Court' in line:
            part = 5

    # Questioning starts at the first mention of 'Q  ' or 'Q.  '
    #if (not started) and any(re.match(q, line) for q in qs[:-1]):
    #    started = 1
    #    print(started)
    
    if started:
        # 1) skip line if line is a) empty, b) page number, or c) confidential statement
        # 2) if answer, set cur_speaker to 1 **A check before Q check to override ":  " match
        # 3) if question, set cur_speaker to 0 and append a new sublist
        if (line == "") or (line.isdecimal()) or (any(re.match(eye, line) for eye in eyes)):
            continue
        
        split = re.split('(' + '|'.join(qs + ans) + ')', line)
        for s in split:
            if any(re.match(a, s) for a in ans):
                cur_speaker = 1
            elif any(re.match(q, s) for q in qs):
                cur_speaker = 0
                out_list.append([part_dict[part],"",""])
            
            # Append line to output
            out_list[-1][cur_speaker + 1] = out_list[-1][cur_speaker + 1] + s + " "

        # Do not capture index
        if "Deposition ended at" in line:
            break

1
1


In [28]:
with open('{}.csv'.format(outpath), 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerows(out_list)